<a href="https://colab.research.google.com/github/Mrprey/UFAL/blob/master/AS6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

O DataSet é sobre imagens de raio-x para detecção de pneumunia
com mais de 50000 imagens para treino e mais de 500 para o teste

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import cv2 # opencv
import matplotlib.pyplot as plt
import os

from sklearn.metrics import accuracy_score

from keras import preprocessing
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop

# Lendo as imagens

Exemplo de imagem

In [3]:
img = cv2.imread('/content/drive/My Drive/Colab Notebooks/DataSets/chest_xray/test/NORMAL/IM-0001-0001.jpeg')

imagem = cv2.resize(img, (19, 29))

imagem



array([[[ 29,  29,  29],
        [ 27,  27,  27],
        [ 26,  26,  26],
        ...,
        [ 28,  28,  28],
        [ 28,  28,  28],
        [ 32,  32,  32]],

       [[ 30,  30,  30],
        [ 24,  24,  24],
        [ 23,  23,  23],
        ...,
        [ 27,  27,  27],
        [ 27,  27,  27],
        [ 29,  29,  29]],

       [[ 31,  31,  31],
        [ 20,  20,  20],
        [ 14,  14,  14],
        ...,
        [ 48,  48,  48],
        [ 18,  18,  18],
        [ 26,  26,  26]],

       ...,

       [[ 25,  25,  25],
        [ 26,  26,  26],
        [ 22,  22,  22],
        ...,
        [187, 187, 187],
        [ 27,  27,  27],
        [ 32,  32,  32]],

       [[ 26,  26,  26],
        [ 28,  28,  28],
        [ 15,  15,  15],
        ...,
        [181, 181, 181],
        [ 27,  27,  27],
        [ 34,  34,  34]],

       [[ 27,  27,  27],
        [ 26,  26,  26],
        [ 18,  18,  18],
        ...,
        [160, 160, 160],
        [ 27,  27,  27],
        [ 31,  31,  31]]

In [4]:
input_dir = '/content/drive/My Drive/Colab Notebooks/DataSets/chest_xray/'

#tipo = train ou test
classes = {'NORMAL':0,'PNEUMONIA':1}
def ler_dataset(tipo):
  X = []
  y = []
  qtd = 5000 if tipo == 'train' else 500
  #para cada pasta
  for pasta in os.listdir(input_dir+tipo+'/'):
    print(pasta)
    #para cada imagem na pasta
    for img_file in os.listdir(input_dir+tipo+'/'+pasta+'/')[:qtd]:
      #leia a imagem
      img = cv2.resize(cv2.imread(input_dir+tipo+'/'+pasta+'/'+img_file), (96, 96))
      #transforme imagem em array de pixels e salve na lista de imagens
      X.append(img)
      #guarde a classificação da imagem
      y.append(classes[pasta])
  X = np.asarray(X)
  y = np.asarray(y)
  return X,y


Lendo Dataset de treino

In [5]:
X_treino,y_treino = ler_dataset('train')

PNEUMONIA
NORMAL


In [6]:
X_treino.shape

(5216, 96, 96, 3)

In [7]:
y_treino.shape

(5216,)

Lendo dataset de test

In [8]:
X_teste,y_teste = ler_dataset('test')

PNEUMONIA
NORMAL


In [9]:
X_teste.shape

(624, 96, 96, 3)

In [10]:
#transformando labels de numérico para categórico
y_treino = to_categorical(y_treino, num_classes = 2)
y_teste = to_categorical(y_teste, num_classes = 2)

# Re-escala dos dados

In [11]:
X_treino = X_treino/255
X_teste = X_teste/255

# Definindo CNN

In [27]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (6,6),padding = 'Same', activation ='relu', input_shape = (96,96,3)))
model.add(Conv2D(filters = 32, kernel_size = (6,6),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 32, kernel_size = (6,6),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())#converte os mapas de ativação finais para um veter de 1 dimensão
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(2, activation = "softmax"))

In [28]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [29]:
# Compilando o modelo
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

# Treinando

In [32]:
# treinamento do modelo

model.fit(X_treino, y_treino, epochs=1, verbose = 1)

163/163 [==============================] - 1015s 6s/step - loss: 6.8150 - accuracy: 0.7868


# Avaliando modelo e resultado

In [33]:
model.evaluate(X_teste,y_teste)

20/20 [==============================] - 26s 1s/step - loss: 0.8971 - accuracy: 0.7388


[0.8971028327941895, 0.7387820482254028]

resultado não foi tão bom, onde um estudo melhor sobre como funciona os parâmetros para o treinamento do modelo poderia ajudar e mas tempo para testes. Com este resultado não seria recomendado colocar este modelo em produção, já que a taxa de erro é considerada alta, principalmente em casos envolvendo risco humano.